In [1]:
# hydrological packages
import hydroeval as he
from hydrotools.nwm_client import utils 

# basic packages
import numpy as np
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
import bz2file as bz2

# system packages
from progressbar import ProgressBar
from datetime import datetime, date
import datetime
import pickle as pkl
import warnings
warnings.filterwarnings("ignore")
import platform
import time

# data analysi packages
from scipy import optimize
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# deep learning packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable


#Shared/Utility scripts
import sys
import boto3
import s3fs
sys.path.insert(0, '../..')  #sys allows for the .ipynb file to connect to the shared folder files
from shared_scripts import lstm_dataprocessing, Simple_Eval, dataloader, lstm_model

#load access key
HOME = os.path.expanduser('~')
KEYPATH = "NWM_ML/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'streamflow-app-data'
BUCKET = S3.Bucket(BUCKET_NAME)

#s3fs
fs = s3fs.S3FileSystem(anon=False, key=ACCESS['Access key ID'][0], secret=ACCESS['Secret access key'][0])

In [6]:
modelname = 'LSTM'
model_path = f"{HOME}/NWM_ML/Model/{modelname}"

cfsday_AFday = 1.983

#input columns
input_columns =[
                'Lat', 
                'Long', 
                'Drainage_area_mi2', 
                'Mean_Basin_Elev_ft',       
                'Perc_Forest', 
                'Perc_Develop', 
                'Perc_Imperv', 
                'Perc_Herbace',       
                'Perc_Slop_30', 
                'Mean_Ann_Precip_in', 
                's1',       
                's2', 
                'storage', 
                'swe', 
                'NWM_flow', 
                'DOY', 
                'tempe(F)', 
                'precip(mm)'
                ]

target = ['flow_cfs']


test_years = [2019, 2020]                 

#load data
datapath = f"{HOME}/NWM_ML/Data/input"
trainingfile = "final_input.parquet"

df, StreamStats = dataloader.get_ML_Data(datapath, trainingfile)
df.head()

df needs no processing


,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,...,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY,tempe(F),precip(mm)
0,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-28,78.55521,-0.891007,-0.453991,0.0,1.2,55.0,301,39.239582,0.0
1,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-29,98.61146,-0.891007,-0.453991,0.0,1.2,55.0,302,45.068712,0.0
2,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-30,97.60208,-0.891007,-0.453991,0.0,1.1,54.0,303,50.945891,0.0
3,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-31,99.33125,-0.891007,-0.453991,0.0,1.2,54.0,304,45.480097,0.0
4,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-11-01,95.76354,-0.998630,0.052336,0.0,1.2,54.0,305,46.656777,0.0


## Dataprocessing
* Editing the features based on the feature importance
* Remove headwater stations from dataset
* make sure dates are in datetime format

### Data scaling

Scaling your data for ML is done for all types of applications and helps the model converge faster. 
If there is no scaling performed, the model will essentially be forced to think certain features are more important than others, rather than being able to learn those things.

There are different ways you can scale the data, such as min-max or standard scaling; both of which are applicable for your model. 
If you know you have a fixed min and max in your dataset (e.g. images), you can use min-max scaling to fix your input and/or output data to be between 0 and 1.

For other applications where you do not have fixed bounds, standard scaling is useful.
This gives all of your features zero-mean and unit variance. Therefore, the distributions of inputs and/or outputs are the same, and the model can treat them as such.

The scaling for your outputs is important in defining the activation function for the output layer.
If you have min-max scaled outputs, you can use sigmoid, because it bounds the outputs to between 0 and 1. 
If you are using standard scaling for the outputs, you would want to be sure you use a linear activation function, because technically standard-scaled outputs are not bounded. 
The choice of output activation is important, and knowledge of how your outputs are scaled is important in determining which activation to use.

In [13]:
#get non headwater stations
headwater_stations = ['10011500', # Bear River headwaters before WY state line
                      '10109000', # Logan River above dams
                      '10113500', # HW Blacksmith fork
                      '10128500', # Upper Weber above Oakley
                      '10131000', #Chalk creek before Weber - lots of upstream irrigation, potentially include
                        '10146400', #Currant Creek above Mona Reservoir - lots of upstream irrigation, potentially include
                        '10150500', #Spanish fork after diamond fork - potentially include because of 6th water diversion CUP
                        '10154200', #Upper Provo river after confluence of N/S forks - potentially include because of duchense tunnel water diversion CUP
                        '10172700', #Vernon creek 2 ranges west of Utah Lake, shouldnt be included because not in GSL basin 
                        '10172800', #Willow creek west of Gransville,  shouldnt be included because does not make it to GSL
                          '10172952'
                          ] #Dunn creek in Raft River Range, shouldnt be included because drains to bonnevile salt flats 

#remove headwater stations
df = df[~df['station_id'].isin(headwater_stations)]

#get stations with correct swe and storage features
#The following sites have swe 

'''
['10011500', '10105900', '10109000', '10126000', '10131000',
       '10133650', '10133800', '10133980', '10134500', '10136500',
       '10140700', '10141000', '10150500', '10154200', '10155000',
       '10155200']
'''

#the following sites have swe and storage
'''
['10126000', '10134500', '10136500', '10140700', '10141000',
       '10155200']
'''

stations = df['station_id'][(df['swe']>0) & (df['storage']>0)].unique()

#Train model with these stations
df = df[df['station_id'].isin(stations)]

#convert dates to datetime format
df.datetime = pd.to_datetime(df.datetime)

# #reset index to clean up df
df.reset_index( inplace =  True, drop = True)

#Set lookback period - This will need to be a grid search variable
lookback = 10
scalertype = 'MinMax'
#add scaler option - minmax standard
x_train_scaled_t, X_test_dic, y_train_scaled_t, y_test_dic = lstm_dataprocessing.Multisite_DataProcessing(df, 
                                                                                   input_columns, 
                                                                                   target, 
                                                                                   lookback, 
                                                                                   test_years, 
                                                                                   model_path,
                                                                                   scalertype) 
# maybe adjust to have x - timesteps in a row, way to only use lookback for timeseries changes (and different time series, e.g., stream/snow),
#Different dataframe set up for static catchment features 

print(x_train_scaled_t.shape, y_train_scaled_t.shape)

torch.Size([26029, 10, 18]) torch.Size([26029])


## Train the model
* make the model a .py file and class when finalized. PyTorch only saves the weights of the layer/node, not the overall structure.

In [23]:
#Train the model
# Assuming you have your data loaded into NumPy arrays as x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled, x_scaled, y_scaled
# Hyperparameters
epochs = 5 #early stopping - Savalan is lookin into it
batch_size = 65
learning_rate = 0.00001 #initial rate
decay = 0.0005 #reduces to 
validation_split = 0.2  #can add cross validation - Savalan is looking into it.
neurons = 300
shuffle = True
bidirectional = True
input_shape = x_train_scaled_t.shape[2]

training_params = epochs, batch_size, learning_rate, decay, neurons, bidirectional
model_params = bidirectional, input_shape, neurons

lstm_model.LSTM_train(training_params,
            x_train_scaled_t,
            y_train_scaled_t, 
            shuffle, 
            model_path,
            modelname)

Preds_Dict = lstm_model.LSTM_predict(model_params, 
                        test_years, 
                        df, 
                        X_test_dic, 
                        input_shape, 
                        StreamStats, 
                        model_path, 
                        modelname)

#Evaluate model performance of the different models, 'flow_cfs_pred', 
prediction_columns = ['NWM_flow', f"{modelname}_flow"]
Eval_DF = Simple_Eval.Simple_Eval(Preds_Dict, 
                                prediction_columns, 
                                modelname, 
                                supply = supply,
                                plots = False, 
                                keystats = False        
                                )

#create dataframe to store key model perf metrics, and inputs
cols = [f"{modelname}_flow_kge", f"{modelname}_flow_rmse", f"{modelname}_flow_mape", f"{modelname}_flow_pbias"]
model_eval = Eval_DF[cols].copy()

#Get mean scoring metrics for AOI - aver kge, mape, pbias
model_eval = pd.DataFrame(model_eval.mean(axis=0)).T


Epoch 1/5, Loss: 0.025307490539149453
Epoch 2/5, Loss: 0.024108693075651985
Epoch 3/5, Loss: 0.02405263412688067
Epoch 4/5, Loss: 0.0240680480775356
Epoch 5/5, Loss: 0.02403112902234023
finish
Run Time: 6.189437627792358 seconds 


In [59]:
import importlib
importlib.reload(lstm_model)

Device: cuda


<module 'shared_scripts.lstm_model' from '/home/rjohnson18/NWM_ML/RJ/ModelDevelopment/../../shared_scripts/lstm_model.py'>

In [60]:
#Train the model
# Assuming you have your data loaded into NumPy arrays as x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled, x_scaled, y_scaled
# parameters
epochs = [10] #early stopping - Savalan is lookin into it
batch_size = [60,80]
learning_rate = [0.0001] #initial rate
decay = [0.005] #reduces to 
neurons = [100]
num_layers = [1]
shuffle = [True]
bidirectional = [True]
input_shape = x_train_scaled_t.shape[2]
supply = False
loss_func = nn.MSELoss()

training_params = epochs, batch_size, learning_rate, decay, neurons, num_layers, bidirectional
model_params = bidirectional, input_shape, neurons, num_layers

GS_Eval_DF, GS_Eval_dict = lstm_model.LSTM_optimization(training_params,
                            loss_func,
                            x_train_scaled_t,
                            y_train_scaled_t, 
                            shuffle,  
                            model_path,
                            modelname,
                            model_params, 
                            test_years, 
                            df, 
                            X_test_dic, 
                            input_shape, 
                            StreamStats,
                            supply)

GS_Eval_DF

Optimizing the LSTM model by evaluating 2 models using grid search validation
Training 1 of 2 models
Parameters: (10, 60, 0.0001, 0.005, 100, 1, True)
60


Epochs completed:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10, Loss: 0.004690202174081643
Epoch 2/10, Loss: 0.004374723489453141
Epoch 3/10, Loss: 0.004375076788697019
Epoch 4/10, Loss: 0.0043720846798824585
Epoch 5/10, Loss: 0.00437717598589075
Epoch 6/10, Loss: 0.004373149613247356
Epoch 7/10, Loss: 0.004378744148826849
Epoch 8/10, Loss: 0.0043730469823445835
Epoch 9/10, Loss: 0.004374976375988955
Epoch 10/10, Loss: 0.004375041546573657
finish
Run Time: 9.191747665405273 seconds 


,NWM_flow_kge,LSTM_flow_kge,NWM_flow_rmse,LSTM_flow_rmse,NWM_flow_mape,LSTM_flow_mape,NWM_flow_pbias,LSTM_flow_pbias
station_id,,,,,,,,
10155200,0.63,-0.58,192.0,307.0,29.28,58.28,15.23,19.07
10140700,0.29,-0.70,236.0,277.0,221.61,271.39,-30.30,-11.76
10136500,-0.42,-0.75,644.0,441.0,330.76,128.16,-137.08,38.20
10141000,-2.10,-0.75,1169.0,526.0,1111.22,164.51,-304.72,30.54
10126000,-0.31,-1.05,1716.0,1452.0,306.30,70.30,-39.65,81.69
10134500,-0.97,-2.02,132.0,170.0,545.15,1071.86,-173.33,-266.82


Training 2 of 2 models
Parameters: (10, 80, 0.0001, 0.005, 100, 1, True)
80


Epochs completed:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10, Loss: 0.004785245909545559
Epoch 2/10, Loss: 0.004370355443847566
Epoch 3/10, Loss: 0.0043696581448165505
Epoch 4/10, Loss: 0.004364768752365436
Epoch 5/10, Loss: 0.004370675746767862
Epoch 6/10, Loss: 0.004367977150868013
Epoch 7/10, Loss: 0.004369670668748304
Epoch 8/10, Loss: 0.004364942609721029
Epoch 9/10, Loss: 0.004368402069203121
Epoch 10/10, Loss: 0.004365497969638733
finish
Run Time: 7.745797157287598 seconds 


,NWM_flow_kge,LSTM_flow_kge,NWM_flow_rmse,LSTM_flow_rmse,NWM_flow_mape,LSTM_flow_mape,NWM_flow_pbias,LSTM_flow_pbias
station_id,,,,,,,,
10155200,0.63,-0.62,192.0,307.0,29.28,59.81,15.23,17.87
10140700,0.29,-0.66,236.0,278.0,221.61,279.49,-30.30,-14.57
10141000,-2.10,-0.68,1169.0,525.0,1111.22,167.98,-304.72,29.22
10136500,-0.42,-0.73,644.0,440.0,330.76,130.04,-137.08,37.27
10126000,-0.31,-0.96,1716.0,1450.0,306.30,70.24,-39.65,81.57
10134500,-0.97,-2.14,132.0,175.0,545.15,1103.95,-173.33,-276.00


,LSTM_flow_kge,LSTM_flow_rmse,LSTM_flow_mape,LSTM_flow_pbias,Epochs,Batchsize,LR,Decay,Neurons,Bidirectional
0,-0.965,529.166667,301.918333,-20.773333,10,80,0.0001,0.005,100,True
1,-0.975,528.833333,294.083333,-18.180000,10,60,0.0001,0.005,100,True


# Finalize automatic training protocol for final model.
Make a prediction for each location, save as compressed pkl file, and send predictions to AWS for use in CSES

In [ ]:
#convert the MLP final model to lstm

Eval_DF, Preds_Dict = mlp_model.Final_Model(GS_Eval_DF,
                x_train_scaled_t,
                y_train_scaled_t, 
                loss_func, 
                model_path, 
                modelname,
                test_years, 
                stations, 
                x_test_temp,
                x_test_scaled, 
                y_test_temp,
                StreamStats,
                station_index_list)
Eval_DF